In [ ]:
import requests
import json
from typing import Iterator, Optional

def chat_stream(message: str, url: str = "http://fastapi:8080/chat/ask", cookie_file: str = "cookies.tmp.txt") -> Iterator[str]:
    """
    Send a chat message and stream the response
    """
    # Load existing cookies if present
    cookies = {}
    try:
        with open(cookie_file, 'r') as f:
            for line in f:
                if line.startswith('#'):
                    continue
                fields = line.strip().split('\t')
                if len(fields) >= 7:
                    cookies[fields[5]] = fields[6]
    except FileNotFoundError:
        pass

    headers = {
        'Content-Type': 'application/json'
    }
    
    data = {
        'message': message
    }

    response = requests.post(
        url,
        headers=headers,
        json=data,
        cookies=cookies,
        stream=True
    )
    
    # Save updated cookies
    with open(cookie_file, 'w') as f:
        for cookie in response.cookies:
            f.write(f"localhost\tFALSE\t/\tFALSE\t0\t{cookie.name}\t{cookie.value}\n")
    
    # Stream the response
    for chunk in response.iter_content(chunk_size=None, decode_unicode=True):
        if chunk:
            yield chunk

In [ ]:
def chat_sync(message: str) -> None:
    """
    Send a chat message and print the response as it arrives
    """
    for chunk in chat_stream(message):
        print(chunk, end='', flush=True)

In [ ]:
chat_sync("Who are you?")